**Task -1 CODSFOT**

**Movie Genre Classification project**

Dataset : https://www.kaggle.com/datasets/hijest/genre-classification-dataset-imdb

### Importing required libraries

In [20]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
import re
import joblib
from sklearn.model_selection import train_test_split

In [21]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CODSOFT/Genre Classification Dataset/train_data.txt'

### Loading the training data :

In [22]:
training_data = pd.read_csv(filepath_or_buffer= DATA_PATH , delimiter=" ::: " , names=["ID" , "Movie" , "Genre" , "Description"])
training_data["Description_Clean"] = training_data["Description"].map(lambda sentence : re.sub(r'\d' , '', sentence))

<ipython-input-22-c00586e87015>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  training_data = pd.read_csv(filepath_or_buffer= DATA_PATH , delimiter=" ::: " , names=["ID" , "Movie" , "Genre" , "Description"])


In [23]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 54214 non-null  int64 
 1   Movie              54214 non-null  object
 2   Genre              54214 non-null  object
 3   Description        54214 non-null  object
 4   Description_Clean  54214 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [24]:
training_data.head()

,ID,Movie,Genre,Description,Description_Clean
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...,Listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous re...,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...,The film's title refers not only to the un-rec...


### Training and validation split:
* Training data : 80%
* Validation data : 20%

In [6]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(list(training_data["Description_Clean"]), list(training_data["Genre"]), test_size=0.2, random_state=42)

### Importing Bert tokenizer from transformers to tokenize plot of the movies:

In [7]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode sequences
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=128, return_tensors='tf')

### Encoding categorical labels to numeric labels , here we have around 27 different classes :

In [8]:
from sklearn.preprocessing import LabelEncoder

# Convert genre labels to numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
num_classes = len(label_encoder.classes_)

In [9]:
num_classes

27

### Importing the TFBertForSequenceClassification model from transformers.
* Here I have used the **bert-base-uncased** model. All the text is lowercased implicitly by the BertTokenizer.
* The optimizer used is Adam.
* The model is compiled with a learning rate of 2e-5.

In [10]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training the model :

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

history = model.fit(
    train_encodings['input_ids'], y_train_encoded,
    validation_data=(val_encodings['input_ids'], y_val_encoded),
    epochs=5,
    batch_size=64,
    callbacks=[early_stopping]
)

Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
678/678 [==============================] - 1307s 2s/step - loss: 1.6304 - accuracy: 0.5471 - val_loss: 1.2496 - val_accuracy: 0.6377
Epoch 2/5
678/678 [==============================] - 1214s 2s/step - loss: 1.1013 - accuracy: 0.6788 - val_loss: 1.1547 - val_accuracy: 0.6577
Epoch 3/5
678/678 [==============================] - 1162s 2s/step - loss: 0.8535 - accuracy: 0.7516 - val_loss: 1.2145 - val_accuracy: 0.6463
Epoch 4/5
678/678 [==============================] - 1214s 2s/step - loss: 0.6240 - accuracy: 0.8189 - val_loss: 1.3050 - val_accuracy: 0.6453


### Saving the trained model :

In [12]:
model.save_weights("model_early_5", save_format="tf")

## Evaluation of the trained model over testing data :

In [13]:
TEST_DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CODSOFT/Genre Classification Dataset/test_data.txt'
TEST_SOLUTION_PATH = '/content/drive/MyDrive/Colab Notebooks/CODSOFT/Genre Classification Dataset/test_data_solution.txt'

testing_data = pd.read_csv(filepath_or_buffer= TEST_DATA_PATH , delimiter=" ::: " , names=["ID" , "Movie" , "Description"])
solution = pd.read_csv(filepath_or_buffer= TEST_SOLUTION_PATH , delimiter=" ::: " , names=["ID" , "Movie" , "Genre" , "Description", ])

testing_data["Description_Clean"] = testing_data["Description"].map(lambda sentence : re.sub(r'\d' , '', sentence))
y_test = solution["Genre"]

<ipython-input-13-ba5d78f608fc>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  testing_data = pd.read_csv(filepath_or_buffer= TEST_DATA_PATH , delimiter=" ::: " , names=["ID" , "Movie" , "Description"])
<ipython-input-13-ba5d78f608fc>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  solution = pd.read_csv(filepath_or_buffer= TEST_SOLUTION_PATH , delimiter=" ::: " , names=["ID" , "Movie" , "Genre" , "Description", ])


In [14]:
testing_data.head()

,ID,Movie,Description,Description_Clean
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apart...","L.R. Brane loves his life - his car, his apart..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty chi...","Spain, March : Quico is a very naughty child o..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family o...,One year in the life of Albin and his family o...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with his...","His father has died, he hasn't spoken with his..."
4,5,Er nu zhai (1955),Before he was known internationally as a marti...,Before he was known internationally as a marti...


In [15]:
test_encodings = tokenizer(list(testing_data["Description_Clean"]), truncation=True, padding=True, max_length=128, return_tensors='tf')

In [16]:
y_test_encoded = label_encoder.transform(y_test)

In [17]:
model.evaluate(test_encodings['input_ids'], y_test_encoded, batch_size = 64)

847/847 [==============================] - 449s 529ms/step - loss: 1.3070 - accuracy: 0.6454


[1.3070216178894043, 0.6453689932823181]